객체지향프로그래밍, file IO, 예외처리, 실행시간, 정규식, matplotlib, numpy, pandas등 library를 이용하여 구현

투수와 타자 정보 데이터 작성

[파일 내 형식]
- 투수 (한 줄 당)
    이름 구단 {구종 : 구사율} ERA AVG BB WHIP
    구단 #(선수) // 이 밑으로는 선수 인원수에 맞게 줄이 쭉 있는데 각 줄에 선수에 대한 정보
    
- 타자

  

입력 :
1. 투수의 이름,구단 입력 -> '이닝 시작' 출력 후 투수의 구종별 구사율 출력
2. 타자의 정보 입력 -> 타자의 타율, 출루율, 변화구/직구 선택률 등 정보 출력 -> 투수의 첫 구종 예상
3. trading start 입력 시 -> '1. 이적\n2. 방출' 출력 후 선택 요구
       1: 이적할 선수의 포지션, 이름, 등번호, 연봉 입력 요구 -> 이름은 다른데 등 번호가 겹칠 경우 '이미 등록된 번호입니다.' -> 이름과 등번호가 겹칠 경우 '이미 등록된 선수입니다.' 출력 ->'등록을 완료했습니다.' 출력
       2: 방출할 선수의 포지션, 이름, 등번호 -> 있다면 '정말 방출하시겠습니까?' 출력 후 y 입력 시 방출

In [17]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd

class BaseballApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Final Project")
        self.geometry("600x450")

        self.pitcher_info = None   # {'name': ..., 'team': ..., 'stats': {...} }
        self.batter_info = None    # {'name': ..., 'team': ..., 'stats': {...} }

        self.create_main_menu()

    def clear_window(self):
        for widget in self.winfo_children():
            widget.destroy()

    def create_main_menu(self):
        self.clear_window()
        tk.Label(self, text="메인 메뉴", font=("Arial", 14)).pack(pady=10)

        tk.Button(self, text="1. 이닝 시작", command=self.create_inning_menu).pack(pady=5)
        tk.Button(self, text="2. 트레이드 시스템", command=self.trade_system).pack(pady=5)
        tk.Button(self, text="3. 프로그램 종료", command=self.exit_program).pack(pady=5)

    def create_inning_menu(self):
        self.clear_window()
        tk.Label(self, text="1. 이닝 시작", font=("Arial", 14)).pack(pady=10)

        # 1. 투수
        if self.pitcher_info:
            tk.Button(self, text="1. 투수 정보 출력", command=self.display_pitcher_info).pack(pady=5)
        else:
            tk.Button(self, text="1. 투수 정보 입력", command=self.enter_pitcher_info).pack(pady=5)

        # 2. 타자
        if self.batter_info:
            tk.Button(self, text="2. 타자 정보 출력", command=self.display_batter_info).pack(pady=5)
        else:
            tk.Button(self, text="2. 타자 정보 입력", command=self.enter_batter_info).pack(pady=5)

        # 3. 예상 결과 분석 (조건부)
        if self.pitcher_info and self.batter_info:
            tk.Button(self, text="3. 예상 결과 분석", command=self.analyze_result).pack(pady=5)
            offset = 1
        else:
            offset = 0

        # 돌아가기 / 종료
        tk.Button(self, text=f"{3 + offset}. 이전 메뉴로 돌아가기", command=self.create_main_menu).pack(pady=5)
        tk.Button(self, text=f"{4 + offset}. 프로그램 종료", command=self.exit_program).pack(pady=5)

    def enter_pitcher_info(self):
        self.open_input_popup("투수 정보 입력", self.save_pitcher_info)

    def enter_batter_info(self):
        self.open_input_popup("타자 정보 입력", self.save_batter_info)

    def open_input_popup(self, title, save_callback):
        popup = tk.Toplevel(self)
        popup.title(title)
        popup.geometry("250x150")

        tk.Label(popup, text="이름:").pack()
        name_entry = tk.Entry(popup)
        name_entry.pack()

        tk.Label(popup, text="소속 구단:").pack()
        team_entry = tk.Entry(popup)
        team_entry.pack()

        def save():
            name = name_entry.get().strip()
            team = team_entry.get().strip()
            if name and team:
                save_callback(name, team)
                popup.destroy()
                self.create_inning_menu()
            else:
                messagebox.showwarning("입력 오류", "모든 정보를 입력해주세요.")

        tk.Button(popup, text="저장", command=save).pack(pady=10)

    def save_pitcher_info(self, name, team):
        try:
            df = pd.read_excel("pitcher.xlsx", engine="openpyxl")
            row = df[(df['Name'] == name) & (df['Team'] == team)]

            if row.empty:
                messagebox.showerror("오류", f"{name} ({team}) 정보가 데이터에 없습니다.")
                return

            stat_cols = ['ERA', 'G', 'W', 'L', 'SV', 'HLD', 'WPCT', 'IP',
                         'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'WHIP']
            stats = {col: row.iloc[0][col] for col in stat_cols}

            self.pitcher_info = {'name': name, 'team': team, 'stats': stats}
            messagebox.showinfo("저장 완료", f"{name} ({team}) 투수 정보가 저장되었습니다.")
        except FileNotFoundError:
            messagebox.showerror("파일 없음", "pitcher.xlsx 파일을 찾을 수 없습니다.")
        except Exception as e:
            messagebox.showerror("에러", f"예기치 않은 오류:\n{e}")

    def save_batter_info(self, name, team):
        try:
            df = pd.read_excel("fielder.xlsx", engine="openpyxl")
            row = df[(df['Name'] == name) & (df['Team'] == team)]

            if row.empty:
                messagebox.showerror("오류", f"{name} ({team}) 정보가 데이터에 없습니다.")
                return

            stat_cols = ['BA', 'H', 'BB', 'SO', 'BB/K', 'AB', 'PA']
            stats = {col: row.iloc[0][col] for col in stat_cols}

            self.batter_info = {'name': name, 'team': team, 'stats': stats}
            messagebox.showinfo("저장 완료", f"{name} ({team}) 타자 정보가 저장되었습니다.")
        except FileNotFoundError:
            messagebox.showerror("파일 없음", "fielder.xlsx 파일을 찾을 수 없습니다.")
        except Exception as e:
            messagebox.showerror("에러", f"예기치 않은 오류:\n{e}")

    def display_pitcher_info(self):
        if not self.pitcher_info:
            messagebox.showwarning("정보 없음", "투수 정보가 없습니다.")
            return

        stats = self.pitcher_info['stats']
        name = self.pitcher_info['name']
        team = self.pitcher_info['team']

        text = f"이름: {name}\n구단: {team}\n\n"
        text += "\n".join([f"{k}: {v}" for k, v in stats.items()])
        messagebox.showinfo("투수 스탯", text)

    def display_batter_info(self):
        if not self.batter_info:
            messagebox.showwarning("정보 없음", "타자 정보가 없습니다.")
            return

        stats = self.batter_info['stats']
        name = self.batter_info['name']
        team = self.batter_info['team']

        text = f"이름: {name}\n구단: {team}\n\n"
        text += "\n".join([f"{k}: {v}" for k, v in stats.items()])
        messagebox.showinfo("타자 스탯", text)

    def analyze_result(self):
        pitcher = self.pitcher_info['stats']
        batter = self.batter_info['stats']

        try:
            so_pitcher = pitcher.get('SO', '?')
            so_batter = batter.get('SO', '?')
            era = pitcher.get('ERA', '?')
            ba = batter.get('BA', '?')

            result = f"""
[예상 결과 분석]

📊 타자 타율(BA): {ba}
⚾ 타자 삼진(SO): {so_batter}

📊 투수 평균자책점(ERA): {era}
⚾ 투수 탈삼진(SO): {so_pitcher}

💡 분석: 타자의 삼진율과 투수의 탈삼진 능력을 참고하여 
결과를 예측해볼 수 있습니다. (시뮬레이션 기능 추가 예정)
"""
            messagebox.showinfo("예상 결과 분석", result.strip())
        except Exception as e:
            messagebox.showerror("분석 오류", f"분석 중 오류 발생:\n{e}")

    def trade_system(self):
        messagebox.showinfo("트레이드 시스템", "트레이드 시스템 실행 중...")

    def exit_program(self):
        self.quit()
        self.destroy()


# 실행
if __name__ == "__main__":
    app = BaseballApp()
    app.mainloop()
